In [37]:
#import the required Python packages
import pandas as pd
import numpy as np

In [38]:
#Note: You might need to add a column for tds level:

scenario = 'drip85_TDS3k'
df=pd.read_csv('drip85_waterdemand.csv') 

In [39]:
tds=pd.read_excel('phase2_tds.xlsx')
tds.set_index('ref',inplace=True)

In [40]:
#Set the default values
for i in range (1,13):
    df['osmotic_pressure'.format(i)] = 0
    df['Edesal_GWh_{}'.format(i)] = 0
df['TDS'] = df['Province'].map(tds['TDS'])

In [41]:
#Define inputs:

osm_coeff = 0.95 #osmotic coefficient =1 
molar_mass = 58440 # molar mass of NaCl = 58.44 mg/mol
dissociation = 2 #NaCL dissociation is to 2 ions Na and Cl
concentration = df['TDS'] 
gas_const = 0.083145  #universal gas constant (R) = 0.083145
wtemp = 25 + 273 #water temperature in Kalven (Tc+273): 25 degree is an assumption
threshold = 3000 #Min TDS content for desalination
#solute_concentration = dissociation * concentration / molar_mass

#osmotic_pressure = osm_coeff * solute_concentration * gas_const * temp

#if (df['tds'] > 2000):
#    return df['osmotic_pressure'] = osm_coeff * (dissociation *df['tds']/molar_mass) *  gas_const * wtemp
#else:
#    return df['osmotic_pressure'] = 0

delta = df.TDS[df['TDS']>threshold] - threshold #the difference between the inlet water quality and the final quality level.

df['osmotic_pressure'] = osm_coeff * (dissociation *delta/molar_mass) *  gas_const * wtemp

df['osmotic_pressure'] = df['osmotic_pressure'].fillna(0)

#df.tds[df['tds']>2000]: This set the criteria that for tds>2000 desalination required

In [42]:
efficiency = 0.85 #overall efficiency
Pf = 55 #feed pressure in bar
Pp = 10 #permeate pressure in bar
Rm = 0.9*(10**14) #mebrane resistance
A = 37 #area m2
N = 15 #number of envelopes
Q = 1
u = 0.00096 #viscosity of water (Ns/m2)

SEC_f = 0.194 #membrane filtration resistance
SEC_R = 0.036 #friction losses, retentate
SEC_p = 0.0016 #friction losses, permeate
SEC_cp = 0.005 #concentration polarization
SEC_inef = 0.068 #pump & ERD inefficiency

In [43]:
#Energy intensity in Kwh/m3

df['Einten_KWh/m3'] = df.loc[df['osmotic_pressure']!=0,'osmotic_pressure']/36 + SEC_f + SEC_R + SEC_p + SEC_cp + SEC_inef
df['Einten_KWh/m3'].fillna(0,inplace=True) 

In [44]:
for i in range(1,13):
    Edesal_GWh = 'Edesal_GWh_{}'.format(i)
    SSWD = 'SSWD_{}'.format(i)
    
    df[Edesal_GWh] = (df['Einten_KWh/m3'] * df[SSWD]) /1000000

In [45]:
#Filter data by provenence: 
df2=df.groupby('NAME_1', as_index=True).sum()
sswd = df2.filter(like='sswd_').sum(axis=1)
Edesal = df2.filter(like='Edesal_GWh_').sum(axis=1)

#final = df2.filter(['area_m2','sswd_','TDS','osmotic_pressure','Edesal_GWh_','country'])
#df2.filter(like='Edesal_GWh_').sum(axis=1)
Edesal

NAME_1
Adrar            0.000000
Biskra           9.418350
Djelfa           0.000000
El Oued        130.430245
Gabes            0.000000
Ghadamis         9.744364
Ghardaia         0.000000
Gharyan          0.000000
Illizi           0.000000
Jufrah           0.000000
Kebili          50.713999
Khenchela        0.000000
Laghouat         0.000000
Musrata         31.336689
Ouargla         69.327688
Tamanrasset      0.000000
Tataouine        3.044172
Tebessa          0.000000
Tozeur           0.000000
dtype: float64

In [46]:
final=df.groupby('NAME_1').sum()
summary = pd.DataFrame({'Irrigated area (ha)':final['area_ha'],
                        'Water demand (m3/ha)':final.filter(like='SSWD_').sum(axis=1)/final['area_ha'],
                        'Edesal (GWh)':final.filter(like='Edesal_GWh_').sum(axis=1)})
summary

,Irrigated area (ha),Water demand (m3/ha),Edesal (GWh)
NAME_1,,,
Adrar,24101.816848,8361.418843,0.000000
Biskra,5749.995666,5311.969611,9.418350
Djelfa,3449.972096,5454.713895,0.000000
El Oued,68999.985344,6072.477958,130.430245
Gabes,2874.983928,5333.373577,0.000000
Ghadamis,5353.213079,5567.014243,9.744364
Ghardaia,34499.998422,6763.319160,0.000000
Gharyan,8557.616421,5727.561501,0.000000
Illizi,1770.396215,7501.553561,0.000000


In [47]:
df['Einten_KWh/m3'].mean()

0.14130494065900223

In [48]:
df.to_csv(scenario + '.csv', index=False)

In [39]:
##Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(scenario + '.xlsx', engine='xlsxwriter')
writer.book.use_zip64()

## Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='All')
#df2.to_excel(writer, sheet_name='Prov')
##sswd.to_excel(writer, sheet_name='SSWD in m3')
summary.to_excel(writer, sheet_name='summary')

## Close the Pandas Excel writer and output the Excel file.
writer.save()